### Imports

In [ ]:
import sys
import cv2 as cv
import re
from scripts import tf_text_graph_common
import tensorflow  as tf
import numpy as np
import matplotlib.pyplot as plt

#from scripts.video_source import VideoSource
from scripts import tf_text_graph_ssd

: 

#### Get the model paths

In [ ]:
inference_graph = "models/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb"
saved_model = "models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/saved_model.pb"
pipeline_config = "models/ssd_mobilenet_v2_coco_2018_03_29/pipeline.config"

In [ ]:
net = cv2.dnn.readNetFromTensorflow(saved_model, pipeline_config)

In [ ]:
## Output path
#config_file = "models/ssd_mobilenet_v2_coco_2018_03_29/saved_model/ssd_mobilenet_v2_coco_2018_03_29.pbtxt"
#
## creat ssd graph
#tf_text_graph_ssd.createSSDGraph(model_file, pipeline_config, config_file)

In [ ]:
# Get class labels
with open("models/ssd_mobilenet_v2_coco_2018_03_29/labels.txt", "r") as f:
    labels = f.read().split("\n")

#### Read the Tensorflow model

In [ ]:
net = cv.dnn.readNetFromTensorflow(model_file, config_file)

In [ ]:
def detect_objects(net, im):
    dim = 300

    blob = cv.dnn.blobFromImage(im, size=(dim, dim), swapRB=True, crop=False)

    net.setInput(blob)

    objects = net.forward()

    return objects

In [ ]:
def display_text(im, text, x, y):
    # Get text size and baseline
    text_size = cv.getTextSize(text, cv.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    dim = text_size[0]
    baseline = text_size[1]
    
    # Draw a filled rectangle for text background
    cv.rectangle(im, (x, y - dim[1] - baseline), (x + dim[0], y+baseline), (255, 255, 255), cv.FILLED)
    
    # Overlay the text on top
    cv.putText(im, text, (x, y - baseline), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv.LINE_AA)

In [ ]:
def display_objects_static(im, objects, threshold=0.25):

    rows = im.shape[0]; cols = im.shape[1]

    for i in range(objects.shape[2]):

        class_id = int(objects[0, 0, i, 1])
        score = float(objects[0, 0, i, 2])

        # Retrieve original coordinates
        x = int(objects[0, 0, i, 3] * cols)
        y = int(objects[0, 0, i, 4] * rows)
        w = int(objects[0, 0, i, 5] * cols) - x
        h = int(objects[0, 0, i, 6] * rows) - y

        # Check if score is above threshold
        if score > threshold:

            # Display bounding box
            display_text(im, f"  {labels[class_id-1]} {round(score,2)}  ", x, y)
            cv.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Convert image to RGB
    mp_img = cv.cvtColor(im, cv.COLOR_BGR2RGB)
    plt.figure(figsize=(30, 10)); plt.imshow(mp_img); plt.axis('off'), plt.show()

In [ ]:
def display_objects_live(im, objects, threshold=0.25):

    rows = im.shape[0]; cols = im.shape[1]

    for i in range(objects.shape[2]):

        class_id = int(objects[0, 0, i, 1])
        score = float(objects[0, 0, i, 2])

        # Retrieve original coordinates
        x = int(objects[0, 0, i, 3] * cols)
        y = int(objects[0, 0, i, 4] * rows)
        w = int(objects[0, 0, i, 5] * cols) - x
        h = int(objects[0, 0, i, 6] * rows) - y

        # Check if score is above threshold
        if score > threshold:

            # Display bounding box
            display_text(im, f"  {labels[class_id-1]} {round(score,2)}  ", x, y)
            cv.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Convert image to RGB
    mp_img = cv.cvtColor(im, cv.COLOR_BGR2RGB)

#### Person tensor

In [ ]:
def crop_persons(im):

    dim = 300

    blob = cv.dnn.blobFromImage(im, size=(dim, dim), swapRB=True, crop=False)

    net.setInput(blob)

    filtered_detections = net.forward()

    # Drop all that are not persons
    detections = filtered_detections[0, 0, :, :]  # Shape: (100, 7)
    filtered_detections = detections[detections[:, 1] == 1]  # Filter class 1 (person)
    print(filtered_detections.shape)
    # Get image dimensions
    rows = im.shape[0]; cols = im.shape[1]

    # Create a list to store cropped images
    crops = []
    for i in range(filtered_detections.shape[0]):
        class_id = int(filtered_detections[i, 1])
        score = float(filtered_detections[i, 2])
        x = int(filtered_detections[i, 3] * cols)
        y = int(filtered_detections[i, 4] * rows)
        w = int(filtered_detections[i, 5] * cols) - x
        h = int(filtered_detections[i, 6] * rows) - y

        # Check if score is above threshold
        if score > 0.5 and class_id == 1:
            crops.append(im[y:y+h//3, x+w//5:x+w-w//5])

        # Get the faces by cropping top third


    return crops
    


In [ ]:
def display_person_live(im, objects, threshold=0.25):

    rows = im.shape[0]; cols = im.shape[1]

    for i in range(objects.shape[2]):

        if int(objects[0, 0, i, 1]) == 1:
            class_id = int(objects[0, 0, i, 1])
            score = float(objects[0, 0, i, 2])

            # Retrieve original coordinates
            x = int(objects[0, 0, i, 3] * cols)
            y = int(objects[0, 0, i, 4] * rows)
            w = int(objects[0, 0, i, 5] * cols) - x
            h = int(objects[0, 0, i, 6] * rows) - y

            # Check if score is above threshold
            if score > threshold:

                # Display bounding box
                display_text(im, f"  {labels[class_id-1]} {round(score,2)}  ", x, y)
                cv.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [ ]:
for i in range(1, 10):

    im = cv.imread(f"image_data/{i}.jpg")
    filtered_detections = detect_objects(net, im)
    #display_objects_static(im, filtered_detections)

#### Webcam inpunt detection 

In [ ]:
## Create video source
#cap = VideoSource(0)
#
## Show video
#win_name = "Object Detection"
#cv.namedWindow(win_name, cv.WINDOW_FREERATIO)
#
#while cv.waitKey(1) < 0:
#    has_frame, frame = cap.read()
#
#    if not has_frame:
#        
#        # release the video capture object
#        cap.release()
#
#        # close all windows
#        cv.destroyAllWindows()
#        break
#
#    objects = detect_objects(net, frame)
#    display_person_live(frame, objects)
#
#    cv.imshow(win_name, frame)

#### Test person cropping

In [ ]:
im = cv.imread("image_data/persons2.jpeg")
crops = crop_persons(im)

for i in range(len(crops)):
    cv.imshow(f"Person {i}", crops[i])
    mp_img = cv.cvtColor(crops[i], cv.COLOR_BGR2RGB)
    plt.figure(figsize=(30, 10)); plt.imshow(mp_img); plt.axis('off'), plt.show()